## Projet 4 - Prét a dépénser
Construction d'une modèle de scoring pour établir si un client peut rembourser ou pas une dette.

Prét à dépenser, une entreprise qui s'occupe des préts pour les particuliers, veut mettre en place une **algorithme de classification** pour aider à décider si un prêt peut-être ou pas accordé à un client.
Pour construire ce modèle nous avons divers fichiers avec notamment :
- un historique des prêts
- un historique d'informations financières
- des informations sur le comportement des emprunteurs.

### Import des libraries

In [ ]:
# import libraries
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from plotly import express as px
import seaborn as sns

from sklearn.preprocessing import LabelEncoder

### Functions


In [ ]:

def show_type(data, type):
    ''' fonction pour afficher les nom de colonnes de type selectionnees 
    Args:
        data: dataframe
        type: type de la colonne à afficher
    '''
    
    obj_col = []
    for columnName in data.columns:
        if data[columnName].dtype == type:
            obj_col.append(columnName)

    # creation d'une df pour stocker les info        
    type_df = pd.DataFrame(obj_col, columns = ["Colonne"])

    # création d'une colonne pour stocker le nombre de valeurs unique dans chaque colonne
    type_df["Valeurs unique"] = type_df["Colonne"].apply(lambda col: data[col].nunique())

    print(type_df)

In [ ]:
def plot_nan(data):
    '''Création d'une barplot des valeurs nulles pour chaque colonne
    Args:
         data: dataframe
    '''
    
    nan_columns = pd.DataFrame({col:{
        # calculs de nombre valeurs nulles totales
        'count' : data[col].isna().sum(),
        # calcul de la pourcentage des valeurs nulles dans la colonne
        'pourc': (data[col].isna().sum()/data.shape[0])*100} for col in data.columns}).transpose()

    # affichage du graphique
    fig = px.bar(nan_columns, color='count', y='pourc',hover_data=['count'],
                 labels=dict(index = 'Colonnes', pourc='Pourcentages valeurs Nan'),
                 title='Pourcentages des valeurs nulles dans le df',
                 width=1000,
                 height=800)
    # ordre des colonnes de plus petit au plus grand nombre des valeurs nulles
    fig.update_layout(xaxis={'categoryorder': 'total ascending'})
    fig.show()

In [ ]:
def pourcentage_nan (data):
    '''afficher la pourcentage des valeurs nulles par colonne
    Args:
        data: dataframe
    '''

    # calcul du total des valeurs manquantes
    total_nan_values = data.isna().sum().sum()
    # calcul du nomobre total des valeurs dans le df
    total_values = data.size
    # calcul de la pourcentage
    pourcentage_vn = round((total_nan_values/total_values)*100,2)

    # affichage des resultats
    print(f'Le nombre total des valeurs manquantes est de: {total_nan_values}')
    print(f'La pourcentage des valeurs manquantes sur le total est de: {pourcentage_vn} %')

In [ ]:
def label_encoder(train_data, test_data):
    ''' encoder les colonnes qui contiennent des valeurs qualitatives
      Args:
         train_data: dataframe avec les valeurs de train
         test_data: dataframe avec les valeurs de test
         '''

    le = LabelEncoder()
    le_count = 0

    # iteration dans chaque colonne
    for col in train_data:
        if train_data[col].dtypes == 'object':
            if len(list(train_data[col].unique())) <= 2:
                # Entrainement sur les donnees train
                le.fit(train_data[col])
                # transformer training et test
                train_data[col] = le.transform(train_data[col])
                test_data[col]=le.transform(test_data[col])

                # comptage de nombre de colonnes modifie
                le_count += 1

    print(f'Le nombre de colonnes label encoded est de {le_count}') 

In [ ]:
def days_employed(data):
    '''Traitement des valeurs précédemment identifiées comme des anomalies dans la colonne DAYS_EMPLOYED
     Args:
         data: dataframe
         '''
    # Calcul des anomalies dans le dataset
    anomalies = data[data['DAYS_EMPLOYED'] == 365243]
    # Affichage des résultats
    print(f"Il y a {len(anomalies)} outliers dans la colonne DAYS_EMPLOYED avant traitement.")
    # On remplace les valeurs aberrantes par des NaN
    data['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace=True)
    # Calcul des anomalies dans le dataset après traitement
    anomalies_apres_traitement = data[data['DAYS_EMPLOYED'] == 365243]
    # Contrôle
    print(f"Il y a {len(anomalies_apres_traitement)} outliers dans la colonne DAYS_EMPLOYED après traitement.\n")

In [ ]:
def calc_corr (data, colonne):
    '''calcul de la correlation entre des colonnes et la colonne des target
    Args:
         data: dataframe
         colonne: colonne du dataframe
         '''
    
    cor_test = data[colonne].corr(train_enc['TARGET'])
    print(f"test de correlation de la colonne {colonne} et 'TARGET' est de {cor_test}")

In [ ]:
def graphique_cat (data, colonne, nomfeat):
    '''creation graphique pour les colonnes qui ont de valeurs categ, et qui vas creer une barplot avec les colonnes divisées par TARGET
    Args: 
        data: dataframe
        colonne: colonne du dataframe
        nomfeat: nom du feature qui va être affiché dans le graphique
        '''
    
    # Création graphique
    fig = px.histogram(
        data,
        x=data[colonne] ,
        opacity=0.7,
        color='TARGET',
        color_discrete_map={0: 'skyblue', 1: 'salmon'},
        title=f'Distribution {nomfeat} divisée par TARGET'
    )
    # Mise à jour du layout
    fig.update_layout(
        xaxis_title=f'{nomfeat}',
        yaxis_title='Frequence',
    )
    # Rendr plus lisible le graphique
    fig.update_traces( opacity=0.9, marker_line=dict(color='black', width=1))
    # Affichage de la figure
    fig.show()

In [ ]:
def graphique_days (data, colonne, feat):
    '''graphique pour les colonnes qui ont des dates, et qui vas creer une barplot avec les colonnes divisées par TARGET
    ARGS: 
        data : dataframe
        colonne : colonne du dataframe
        feat : feature qui va être affiché dans le graphique
    '''
    fig = px.histogram(
        data,
        x=abs(data[colonne] /  365),
        nbins=30,
        opacity=0.7,
        color='TARGET',
        color_discrete_map={0: 'skyblue', 1: 'salmon'},

        title=f'Distribution des {feat} par TARGET'
    )
    # Mise à jour du layout
    fig.update_layout(
        xaxis_title=f'{feat}',
        yaxis_title='Frequence',
    )
    # Rendr plus lisible le graphique
    fig.update_traces( opacity=0.9, marker_line=dict(color='black', width=1))
    # Affichage de la figure
    fig.show()

In [ ]:
def graph_ext(data,colonne):
    '''Function qui affiche un graphique en rélation aux données target
    ARG:
        data : dataframes
        colonne : colonne du dataframe
    '''
    # affichage des dettes payées
    sns.kdeplot(data.loc[data['TARGET'] == 0, colonne], label = 'target == 0')
    # affichage des dettes impayées
    sns.kdeplot(data.loc[data['TARGET'] == 1, colonne], label = 'target == 1')

    plt.show()

In [ ]:
def remove_high_correlation(train_df, test_df, threshold):
    '''Function pour supprimer les colonnes qui ont un correlation trés fortes
    ARGS:
        train_df : datframe train
        test_df : dataframe test
        threshold : pourcentage de correlation'''
    
    # Comptage des colonnes avant le traitement
    num_columns_before = train_df.shape[1]
    # Calcul de la matrice de corrélation
    corr_train = train_df.corr()
    # Récupération des indices des paires de colonnes avec une corrélation supérieure ou égale au seuil
    high_corr_indices = [(i, j) for i in range(len(corr_train)) for j in range(i) if corr_train.iloc[i, j] >= threshold]
    # Récupération des premiers éléments des paires de colonnes fortement corrélées
    high_corr_columns = [corr_train.columns[i] for i, j in high_corr_indices]
    # Suppression des colonnes fortement corrélées du DataFrame train_df
    train_df.drop(columns=high_corr_columns, inplace=True)
    # Suppression des mêmes colonnes du DataFrame test_df
    test_df.drop(columns=high_corr_columns, inplace=True)
    # Comptage des colonnes après le traitement
    num_columns_after = train_df.shape[1]
    # Affichage des colonnes supprimées
    print(f"Colonnes fortement corrélées supprimées : {', '.join(high_corr_columns)}")
    # Affichage du nombre de colonnes avant et après le traitement
    print(f"Nombre de colonnes avant le traitement : {num_columns_before}")
    print(f"Nombre de colonnes après le traitement : {num_columns_after}")

### Import des données

On vas se concentrer sur le fichier ``application_train.csv`` pour commencer à effectuer l'EDA, et puis on import aussi ``application_test`` pour effectuer le mêmes modification que on vas effectuer sur le premier fichier.

In [ ]:
# Import des fichiers
train = pd.read_csv("../data/raw/application_train.csv")
test = pd.read_csv("../data/raw/application_test.csv")

### Analyses générales

On vas effectuer des premières contrôles pour avoir une idée des deux df.

In [ ]:
    # affichage train
train

In [ ]:
# affichage test
test

In [ ]:
# affichage des dimensions des df
print(f'Le dataframe train contient {train.shape[0]} lignes et {train.shape[1]} colonnes')
print(f"Le dataframe test contient {test.shape[0]} lignes et {test.shape[1]}")

In [ ]:
# controle valeurs nulles
pourcentage_nan (train)

In [ ]:
# controle valeurs nulles
pourcentage_nan (test)

On observe la présece de environ 24% des valeurs nulles dans les deux df

In [ ]:
# affichage graphique pour observer les valeurs nulles
plot_nan(train)

In [ ]:
# affichage graphique pour observer les valeurs nulles
plot_nan(test)

In [ ]:
# drop des colonnes qui ont plus de 60% des valeurs nulles
for columnName in train:
    if train[columnName].isna().sum() / len(train[columnName]) > 0.6:
        train.drop(columnName, axis=1, inplace=True)

In [ ]:
# drop des colonnes avec plus de 60 % des valeurs nulles
for columnName in test:
    if test[columnName].isna().sum() / len(test[columnName]) > 0.6:
        test.drop(columnName, axis=1, inplace=True)

In [ ]:
# controle des types
train.dtypes.value_counts()

In [ ]:
# nombre des valeurs positifs et negatifs dans la colonne target
train['TARGET'].value_counts()

In [ ]:
# graphique valeurs TARGET
fig = px.histogram(train, x='TARGET', title='Histogram des valeurs TARGET')
fig.update_xaxes(title_text='Valeurs')
fig.update_yaxes(title_text='Frequence')

# on rend le graphique plus lisible
fig.update_layout(bargap=0.1)
fig.update_traces(marker_color='skyblue', opacity=0.7, marker_line=dict(color='blue', width=1))

# affichage
fig.show()
fig.write_image("../Results/histogram_target.png")

On oberve que la majorité des valeurs contenu dans la variable ''TARGET'' sont des 0, donc en géneral la majorité des clients tend a payer leur debts.

### Traitement des données et encodage

In [ ]:
# affichage des colonnes de type object
train.dtypes.value_counts()

In [ ]:
# affichage des colonnes de type object
show_type(train, object)

In [ ]:
#encodage donnes avec 2 categories
label_encoder(train, test)

In [ ]:
# one-hot encoding variables categorielles
train_enc = pd.get_dummies(train)
test_enc = pd.get_dummies(test)

In [ ]:
# Sélectionnez la colonne TARGET du df_TRAIN
train_labels = train_enc['TARGET']

In [ ]:
# Alignement des données d'entraînement et de test et stockage des colonnes présentes dans le deux df
train_enc, test_enc = train_enc.align(test_enc, join = 'inner', axis = 1)

In [ ]:
# Ajout de la colonne TARGET à nouveaux
train_enc['TARGET'] = train_labels


In [ ]:
# affichage des dimensions des deux df
print('Dimension du train encodé: ', train_enc.shape)
print('Dimension du test encodé: ', test_enc.shape)

In [ ]:
# sauvegarde de df
train_enc.to_csv('../Data/Processed/train_enc.csv', index=False)
test_enc.to_csv('../Data/Processed/test_enc.csv', index=False)

### Traitement des outliers

In [ ]:
# on affiche une describe en transformant les valeurs dans la colonne en années pour les rendre plus lisibles
(train_enc['DAYS_BIRTH'] / - 365).describe()

In [ ]:
# création histogram pour montrer la distribution de l'age
fig = px.histogram(train, x=test['DAYS_BIRTH'] / -365, nbins=30, title='Âge du client')
fig.update_layout(xaxis_title='Age', yaxis_title='Count')

fig.update_layout(bargap=0.1)
fig.update_traces(marker_color='skyblue', opacity=0.7, marker_line=dict(color='blue', width=0.5))


# affichage du graphique
fig.show()
fig.write_image('../Results/Histogram_age.png')

In [ ]:
# on effectue le même controle sur la colonne DAYS_EMPLOYED
train_enc["DAYS_EMPLOYED"].describe()

Le max, representé par 365243 est un valeur aberrante parceque en le divisant par 365 on trouve 1000 années, que on vas traiter avec la fonction days_employed

In [ ]:
days_employed(train_enc)
days_employed(test_enc)
days_employed(train)
days_employed(test)

In [ ]:
# création histogram pour montrer la distribution de jours pour payer les debts
fig = px.histogram(train_enc, x=train_enc['DAYS_EMPLOYED'] / -365, nbins=30, title='annes pour payer une dettes')
fig.update_layout(xaxis_title='Années', yaxis_title='Count')

# update pour rendre le graphique plus lisible
fig.update_layout(bargap=0.1)
fig.update_traces(marker_color='skyblue', opacity=0.7, marker_line=dict(color='blue', width=0.5))


# affichage du graphique
fig.show()
fig.write_image('../Results/anne_payement_dettess.png')

Nous notons qu'en général, la majorité des clients prend environ un à deux ans pour régler leurs dettes

### Analyse correlations

In [ ]:
# recherche des correlation 
correlations = train_enc.corr()['TARGET'].sort_values()

In [ ]:
# affichage correlations positives
print('Correlation plus positives:\n', correlations.tail(10))

On observe des correlations interessantes avec ```DAYS_BIRTH```, ```DAYS_EMPLOYED``` et ```REGION_RATING_CLIENT_W_CITY```

In [ ]:
# affichage correlations negatives
print('\nCorrelations plus negatives:\n', correlations.head(10))

On observe un correlation interessante avec les ```EXT_SOURCE_n```

In [ ]:
# preparation données pour matrice des correlation
ext_data = train_enc[['TARGET', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'REGION_RATING_CLIENT_W_CITY', 'EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']]
ext_data_corrs = ext_data.corr()

In [ ]:
# Heatmap des correlations
plt.figure(figsize = (8, 6))
sns.heatmap(ext_data_corrs, annot = True,cmap='BrBG', vmax = 0.6)
plt.title('Correlation Heatmap');
plt.savefig('../Results/heatmap_correlation_1.png')   

In [ ]:
# creation graphique
calc_corr(train, 'DAYS_BIRTH')
graphique_days(train,'DAYS_BIRTH','Age')
fig.write_image('../Results/distribution_age_target.png')

On observe ici comment les distribution des dettes sont plutot biens distrubuées entre 20 e 60 ans, mais en général on voit que il y a beaucoup des impayée autour des 30 ans.

In [ ]:
# creation graphique
calc_corr(train, 'DAYS_EMPLOYED')
graphique_days(train, 'DAYS_EMPLOYED', 'Années')
fig.write_image('../Results/distribution_anne_target_1.png')

La majoritées des dettes est payée entre 1 et 2 années et on observe aussi un concetration des impayées entre le deuxieme et le troisieme année

In [ ]:
# creation graphique
calc_corr(train_enc, 'REGION_RATING_CLIENT_W_CITY')
graphique_cat(train_enc,'REGION_RATING_CLIENT_W_CITY','note_de_la_region')
fig.write_image('../Results/distribution_not_reg_t.png')

La majorité des credits son't concentrées dans les regions que sont notée 2.

In [ ]:
# creation graphique
graphique_cat(train,'CODE_GENDER',('Genre'))
fig.write_image('../Results/distribution_t_gen.png')

En general, on observe qu'il y a plus femmes que hommes dans la distribution des TARGET du df.

In [ ]:
# creation graphique
calc_corr(train, 'EXT_SOURCE_3')
graph_ext(train, 'EXT_SOURCE_3')
plt.savefig('../Results/ext_s_3_tar.png')

In [ ]:
# creation graphique
calc_corr(train, 'EXT_SOURCE_2')
graph_ext(train, 'EXT_SOURCE_2')
plt.savefig('../Results/ext_s_2_tar.png')

In [ ]:
# creation graphique
calc_corr(train, 'EXT_SOURCE_1')
graph_ext(train, 'EXT_SOURCE_1')
plt.savefig('../Results/ext_s_1_tar.png')

In [ ]:
# creation graphique
graphique_cat(train,'NAME_EDUCATION_TYPE',('niveaux_education'))
fig.write_image('../Results/distr_niv_instr.png')

On voit ici, que ceux qui sont arrivées au niveaux secondary, ont la tendance a demander plus des dettes par rapport aux autres.

In [ ]:
# creation graphique
graphique_cat(train,'NAME_INCOME_TYPE','Poste_travail')
fig.write_image('../Results/distr_trav_1.png')

Ceux qui font partie de la categorie ``working`` ont la tendance a faire plus de detts par rapport aux autres categories, on peut immaginer que celle ci est un categorie plus generale, qui comprend plusieurs categories des professionnels.

### Creation nouvelles variables

In [ ]:
# creation nouveaux variables
df_train = train_enc.copy()
df_test = test_enc.copy()
df_train_nv = train_enc.copy()
df_test_nv = test_enc.copy()

In [ ]:
# Calcul du pourcentage crédit sur revenu
df_train_nv['CREDIT_INCOME_PERCENT'] = train_enc['AMT_CREDIT'] / train_enc['AMT_INCOME_TOTAL']
# Calcul du pourcentage annuité sur revenu
df_train_nv['ANNUITY_INCOME_PERCENT'] = train_enc['AMT_ANNUITY'] / train_enc['AMT_INCOME_TOTAL']
# Calcul de la durée du crédit
df_train_nv['CREDIT_TERM'] = train_enc['AMT_ANNUITY'] / train_enc['AMT_CREDIT']
# Calcul du pourcentage jours employé sur jours de naissance
df_train_nv['DAYS_EMPLOYED_PERCENT'] = train_enc['DAYS_EMPLOYED'] / train_enc['DAYS_BIRTH']

In [ ]:
# Calcul du pourcentage crédit sur revenu
df_test_nv['CREDIT_INCOME_PERCENT'] = train_enc['AMT_CREDIT'] / train_enc['AMT_INCOME_TOTAL']
# Calcul du pourcentage annuité sur revenu
df_test_nv['ANNUITY_INCOME_PERCENT'] = train_enc['AMT_ANNUITY'] / train_enc['AMT_INCOME_TOTAL']
# Calcul de la durée du crédit
df_test_nv['CREDIT_TERM'] = train_enc['AMT_ANNUITY'] / train_enc['AMT_CREDIT']
# Calcul du pourcentage jours employé sur jours de naissance
df_test_nv['DAYS_EMPLOYED_PERCENT'] = train_enc['DAYS_EMPLOYED'] / train_enc['DAYS_BIRTH']

### Supprimer corrélation 

In [ ]:
# supprimer les colonnes qui ont une fort correlation entre eux
remove_high_correlation(df_train, df_test, 0.9)

### Sauvegarde des Dataframes

In [ ]:
# sauvegarde train
df_train.to_csv('../Data/Processed/df_train.csv', index=False)
df_train_nv.to_csv('../Data/Processed/df_train_nv.csv', index=False)

In [ ]:
# sauvegarde test
df_test.to_csv('../Data/Processed/df_test.csv', index=False)
df_test_nv.to_csv('../Data/Processed/df_test_nv.csv', index=False)